Em vez de estar sempre a usar os thresholds de 90/180, usar um slider. Por exemplo quando o slider esta nos 45, ele vai fazer o processamento todo (o pipeline todo) para me dar a imagem que em vez de ser com thresholds de 90/180, seja com 45/90. 

Tenho que começar a agrupar isto por funções, de maneira a que qnd altero o 1º parametro, tem-se um input e esse input vai fazer com que a imagem do canny vá mudar e vamos correr o algoritmo para as imagens em questão

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as nc
import cv2
import datetime
import xarray as xr
import os
plt.rcParams["figure.figsize"] = 10, 8
%matplotlib inline

### Importação dos dados de Atlantic-Iberian Biscay Irish- Ocean Physics Reanalysis

Importar o ficheiro netCDF da pasta data do repositório e depois converte-lo para o formato de dataframa


In [76]:
current_path = os.getcwd()
current_path

'/home/luisfigueiredo/JUNO/notebooks'

In [77]:
from pathlib import Path

data_folder = os.path.join(current_path,"../data")
data_folder

'/home/luisfigueiredo/JUNO/notebooks/../data'

In [78]:
nc_path = os.path.join(data_folder, "IBI2014-2019.nc")
ds = nc.Dataset(nc_path)
data = xr.load_dataset(nc_path)

In [79]:
print(ds)     # informações relativas ao ficheiro netCDF

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    source: CMEMS IBI-MFC
    institution: Puertos del Estado (PdE) - Mercator-Ocean (MO)
    references: http://marine.copernicus.eu
    title: CMEMS IBI REANALYSIS: DAILY PHYSICAL PRODUCTS 
    easting: longitude
    northing: latitude
    domain_name: IBI12
    FROM_ORIGINAL_FILE__field_type: mean
    field_date: 20191224
    FROM_ORIGINAL_FILE__longitude_min: -19.f
    FROM_ORIGINAL_FILE__longitude_max: 5.f
    FROM_ORIGINAL_FILE__latitude_min: 26.f
    FROM_ORIGINAL_FILE__latitude_max: 56.f
    z_min: 0.50576f
    z_max: 5698.061f
    contact: mailto: servicedesk.cmems@mercator-ocean.eu
    bulletin_date: 2020-12-01
    bulletin_type: Reanalysis
    _CoordSysBuilder: ucar.nc2.dataset.conv.CF1Convention
    comment: 
    history: Data extracted from dataset http://localhost:8080/thredds/dodsC/cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m
    dimensions(sizes): depth(1), latitude(1

In [80]:
#Converter o ficheiro netCDF para uma dataframe
datadf = data.to_dataframe()
df2014_2019 = datadf.reset_index()

In [81]:
df2014_2019 = df2014_2019.drop(['depth'], axis=1)     #dropar coluna 'depth' (não é necessária pq é sempre igual)
df2014_2019   #os dados .nc estão agora no formato de uma dataframe

,latitude,longitude,time,thetao
0,35.0,-19.0,2014-01-01 12:00:00,18.316000
1,35.0,-19.0,2014-01-02 12:00:00,18.309000
2,35.0,-19.0,2014-01-03 12:00:00,18.316000
3,35.0,-19.0,2014-01-04 12:00:00,18.273001
4,35.0,-19.0,2014-01-05 12:00:00,18.230000
...,...,...,...,...
44660611,45.0,-5.0,2019-12-20 12:00:00,13.619000
44660612,45.0,-5.0,2019-12-21 12:00:00,13.559000
44660613,45.0,-5.0,2019-12-22 12:00:00,13.458000
44660614,45.0,-5.0,2019-12-23 12:00:00,13.400000


In [82]:
df2014_2019 = df2014_2019[['time', 'latitude', 'longitude', 'thetao']]    #reorganizar as colunas da dataframe
df2014_2019

,time,latitude,longitude,thetao
0,2014-01-01 12:00:00,35.0,-19.0,18.316000
1,2014-01-02 12:00:00,35.0,-19.0,18.309000
2,2014-01-03 12:00:00,35.0,-19.0,18.316000
3,2014-01-04 12:00:00,35.0,-19.0,18.273001
4,2014-01-05 12:00:00,35.0,-19.0,18.230000
...,...,...,...,...
44660611,2019-12-20 12:00:00,45.0,-5.0,13.619000
44660612,2019-12-21 12:00:00,45.0,-5.0,13.559000
44660613,2019-12-22 12:00:00,45.0,-5.0,13.458000
44660614,2019-12-23 12:00:00,45.0,-5.0,13.400000


In [12]:
# Guardar esta df no nosso repositorio nao e grande ideia porque ocupa 2,5GB

### Gerar array com os dias para os quais quero depois aplicar o canny

É criado o dicionário days que tem muitas dataframes e depois dá para aceder a estas df individualmente com os valores do array que são extraidos através do specific day

In [83]:
dataf2014_2019 = df2014_2019.copy()

In [84]:
specificday = [pd.Timestamp(dd).strftime("%Y-%m-%d %H:%M:%S") for dd in dataf2014_2019['time'].unique()]

In [85]:
specificday = np.array(specificday, dtype=np.object)

In [86]:
specificday  

array(['2014-01-01 12:00:00', '2014-01-02 12:00:00',
       '2014-01-03 12:00:00', ..., '2019-12-22 12:00:00',
       '2019-12-23 12:00:00', '2019-12-24 12:00:00'], dtype=object)

In [87]:
#create a dictionary to store the data frames for each day
days = {elem : pd.DataFrame for elem in specificday}

for key in df.keys():
    days[key] = dataf2014_2019[:][dataf2014_2019['time'] == key]

In [119]:
ond2014 = specificday[(specificday>='2014-10-01 12:00:00') & (specificday <= '2014-12-31 12:00:00')]#+(specificday>='2015-10-01 12:00:00') & (specificday <= '2015-12-31 12:00:00') + (specificday>='2016-10-01 12:00:00') & (specificday <= '2016-12-31 12:00:00') + (specificday>='2017-10-01 12:00:00') & (specificday <= '2017-12-31 12:00:00') + (specificday>='2018-10-01 12:00:00') & (specificday <= '2018-12-31 12:00:00') + (specificday>='2019-10-01 12:00:00') & (specificday <= '2019-12-31 12:00:00')]
ond2014

array(['2014-10-01 12:00:00', '2014-10-02 12:00:00',
       '2014-10-03 12:00:00', '2014-10-04 12:00:00',
       '2014-10-05 12:00:00', '2014-10-06 12:00:00',
       '2014-10-07 12:00:00', '2014-10-08 12:00:00',
       '2014-10-09 12:00:00', '2014-10-10 12:00:00',
       '2014-10-11 12:00:00', '2014-10-12 12:00:00',
       '2014-10-13 12:00:00', '2014-10-14 12:00:00',
       '2014-10-15 12:00:00', '2014-10-16 12:00:00',
       '2014-10-17 12:00:00', '2014-10-18 12:00:00',
       '2014-10-19 12:00:00', '2014-10-20 12:00:00',
       '2014-10-21 12:00:00', '2014-10-22 12:00:00',
       '2014-10-23 12:00:00', '2014-10-24 12:00:00',
       '2014-10-25 12:00:00', '2014-10-26 12:00:00',
       '2014-10-27 12:00:00', '2014-10-28 12:00:00',
       '2014-10-29 12:00:00', '2014-10-30 12:00:00',
       '2014-10-31 12:00:00', '2014-11-01 12:00:00',
       '2014-11-02 12:00:00', '2014-11-03 12:00:00',
       '2014-11-04 12:00:00', '2014-11-05 12:00:00',
       '2014-11-06 12:00:00', '2014-11-07 12:0

### Definir funções para todo o pipeline

In [91]:
def canny_creation(days, Tmax):
    
    """
    Função que pega numa df com temperaturas relativas a um certo dia e retorna a matriz obtida através do Canny
    com a mask aplicada
    """
    
    #Pegar na df dum dia e converte-la num np array com os valores de Temperatura para os valores de long e lati
    Temp = days.pivot_table(index='longitude', columns='latitude', values='thetao').T.values
    
    #Converter os valores de temperatura no formato uint8 com valores entre 0-255
    Temp_day = ((Temp - np.nanmin(Temp)) * (1/(np.nanmax(Temp) - np.nanmin(Temp)) * 255)).astype('uint8')

    Temp_day = np.flipud(Temp_day)   #flipud -> Reverse the order of elements along axis 0 (up/down).
    
    #apply the canny algorithm and plot the image with the edges
    canny = cv2.Canny(Temp_day, Tmax/2, Tmax, L2gradient=False, apertureSize=3)
    
    mask = np.isnan(np.flipud(Temp))    #Boolean array é True onde o array original (Temp) tinha valores Nan
    mask255 =np.where(mask,(np.ones(mask.shape))*255,0).astype("uint8")   #array cujos pixels= 255 quando mask=True
    #Fazer uma dilatação para assegurar que os pixeis que pertencem à costa não são considerados como frentes no canny
    kernel = np.ones((3,3), np.uint8)
    mask_dilated = cv2.dilate(mask255, kernel)
    canny =np.ma.masked_where(mask_dilated==255, canny)
    
    return canny

In [92]:
def FrontalProb(period, Tmax):
    """
    Função que retorna um array de shape (121, 169) com a frontal probability de cada pixel
    """
    fp = np.zeros((121,169))
    for day in period:
        fp = fp + canny_creation(days[day], Tmax)
    fp = fp/(len(period)*255)*100
    
    return fp

In [120]:
def visualization(Tmax):
    
    """
    Função para visualizar o map of frontal probability e um histograma com as frontal probabilities
    """
    
    fp = FrontalProb(period=ond2014, Tmax=Tmax)
    
    plt.rcParams["figure.figsize"] = 16, 12
    fig = plt.figure()
    ax1 = fig.add_subplot(2,2,1)
    img = ax1.imshow(fp) 
    plt.colorbar(img, orientation='horizontal', fraction=0.025, pad=0.08, aspect=50)
    plt.title("Map of frontal probability for certain period", fontsize=15)

    ax2 = fig.add_subplot(2,2,2)
    ax2.hist(fp.flatten(), bins=100);
    plt.xlabel('Frontal Probability',fontsize=10)
    plt.ylim([0, 2000]);


In [121]:
#Utilizar widgets (sliders) para ver diferentes visualizações conforme o threshold
import ipywidgets

fp = ipywidgets.interact(visualization, Tmax=(1, 250))

interactive(children=(IntSlider(value=125, description='Tmax', max=250, min=1), Output()), _dom_classes=('widg…